<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>Colaboratory nedir?</h1>

Colaboratory &#40;ya da kısaca "Colab"&#41;, tarayıcınızda Python'u yazmanızı ve çalıştırmanızı sağlar. Üstelik: 
- Hiç yapılandırma gerektirmez
- GPU'lara ücretsiz erişim imkanı sunar
- Kolay paylaşım imkanı sunar

İster <strong>öğrenci</strong> ister <strong>veri bilimci</strong> ister <strong>yapay zeka araştırmacısı</strong> olun, Colab işinizi kolaylaştırabilir. Daha fazla bilgi edinmek için <a href="https://www.youtube.com/watch?v=inN8seMm7UI">Colab'e Giriş</a> videosunu izleyebilir ya da aşağıdan hemen kullanmaya başlayabilirsiniz.

# 1) Importing Python Packages for GAN

In [2]:
from keras.datasets import mnist

from keras.models import Sequential
from keras.layers import BatchNormalization
from keras.layers import Dense, Reshape, Flatten
from keras.layers.advanced_activations import LeakyReLU
from tensorflow.keras.optimizers import Adam

import numpy as np
!mkdir generated_images

# 2) Variables for Neural Networks & Data

In [3]:
# image width and height for mnist handwritten digits
img_width = 28
img_height = 28

# Grayscale is 1 color channel, RGB is 3 color channel
channels = 1 

img_shape = (img_width, img_height, channels)

# 100 random values - noise
latent_dim = 100 

# Adam is an optimizer that implements Adam algorithm, Adam optimization is a stochastic gradiend descent method
# lr - learning rate - how fast learning occurs
adam = Adam(lr=0.0001)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


# 3) Building Generator

In [10]:
def build_generator():
  model = Sequential()

  # adds layer which has 256 output neurons and our noise (latent_dim)
  model.add(Dense(256, input_dim=latent_dim))

  # activation layer
  model.add(LeakyReLU(alpha=0.2))

  # increases stability of GAN
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  # output shape like our input shape 28x28
  model.add(Dense(np.prod(img_shape), activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()
  return model

generator = build_generator()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 256)               25856     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 256)               0         
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_15 (Dense)            (None, 512)               131584    
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 512)               0         
                                                                 
 batch_normalization_10 (Bat  (None, 512)              2048      
 chNormalization)                                     

# 4) Building Discriminator

In [12]:
def build_discriminator():
  model = Sequential()

  # squishes img_shape (output of generator method)
  model.add(Flatten(input_shape=img_shape))
  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dense(256))
  model.add(LeakyReLU(alpha=0.2))
  
  # sonuçları artırmak için ek katmanlar (layer) kullanılabilir ancak modelin çalışması için daha fazla eğitim gerekecektir.
  # model.add(Dense(256))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(Dense(256))
  # model.add(LeakyReLU(alpha=0.2))
  # model.add(Dense(256))
  # model.add(LeakyReLU(alpha=0.2))

  model.add(Dense(1, activation='sigmoid'))

  model.summary()
  return model

discriminator = build_discriminator()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_21 (Dense)            (None, 512)               401920    
                                                                 
 leaky_re_lu_14 (LeakyReLU)  (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 256)               131328    
                                                                 
 leaky_re_lu_15 (LeakyReLU)  (None, 256)               0         
                                                                 
 dense_23 (Dense)            (None, 1)                 257       
                                                                 
Total params: 533,505
Trainable params: 533,505
Non-tr

# 5) Connectin Neural Networks to Build GAN

In [29]:
# loss parameter is used for loss function gradient descent and binary crossentropy is used because our output is binary
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])

GAN = Sequential()
discriminator.trainable = False
GAN.add(generator)
GAN.add(discriminator)

GAN.compile(loss='binary_crossentropy', optimizer='adam')

# 6) Outputting Images

In [30]:
#@title
## **7) Outputting Images**
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name = 0.00000000

def save_imgs(epoch):
  r, c = 5, 5
  noise = np.random.normal(0, 1, (r * c, latent_dim))
  gen_imgs = generator.predict(noise)
  global save_name
  save_name += 0.00000001
  print("%.8f" % save_name)

  # Rescale images 0 - 1
  gen_imgs = 0.5 * gen_imgs + 0.5

  fig, axs = plt.subplots(r, c)
  cnt = 0
  for i in range(r):
    for j in range(c):
      axs[i,j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')
      axs[i,j].axis('off')
      cnt += 1
  fig.savefig("generated_images/%.8f.png" % save_name)
  print('saved')
  plt.close()


# 7) Training GAN

In [31]:
# batch_size bir seferde neural network e girilen imaj sayısı, # 60000 giriş imaj için ideal, 1000 imaj için 16, 30000 imaj içinse 32 kullanılabilir
# save_interval - kayıt aralığı
def train(epochs, batch_size=64, save_interval=200): 
  # verileri yüklediğimizde (x_train, y_train), (x_test, y_test) şeklinde bir yükleme yapılır, 
  # x_train (60000, 28, 28) 60000 adet ve her biri 28x28 olan imajları ifade eder
  # y_train yüklenen imajlara ilişkin etiket (label) verilerini içerir. Buradaki değer mesela x_train içine eklenmiş olan el ile yazılmış (hand written) 1 rakamının 1 olan karşılığı olup,
  # modelimizin hedefi de bu veriyi elde etmektir.
  # x_test ve y_test de test verilerimizdir ancak GAN ile uğraştığımız için bu verilere de ihtiyacımız yoktur
  (X_train, _), (_, _) = mnist.load_data()
  
  # for normalizing our data, this squish it to a small form valued between -1 and 1, bu sayede training işlemi daha performaslı olacaktır.
  X_train = X_train / 127.5 -1.

  # y value that we want from neural network to predict 
  # valid vector has full of (=64 (batch_size)) ones in it
  valid = np.ones((batch_size, 1))
  # vector full of zeroes
  fakes = np.zeros((batch_size, 1))

  for epoch in range(epochs):
    # takes a random image collection (64 image batch)
    idx = np.random.randint(0, X_train.shape[0], batch_size)
    
    # loads 64 random images to imgs
    imgs = X_train[idx]

    # get 64 noisy images
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train The discriminator
    d_loss_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake = discriminator.train_on_batch(gen_imgs, fakes)
    # gerçek imajlarla geçerli (valid) ve sahte imajlarla sahte olduğu (noise) öğretilen discriminator ın kayıp ortalaması bulunur
    d_loss = np.add(d_loss_real, d_loss_fake) * 0.5

    # generator ve discriminator ın aynı gürültü ile üretilmemesi için farklı bir gürültü üretilir, 64 adet latent_dim (gürültü) üretilir 
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Training The GAN, inverse y label
    g_loss = GAN.train_on_batch(noise, valid)

    print("************** %d [D loss: %f, acc: %.2f] [G loss: %f]" % (epoch, d_loss[0], d_loss[1] * 100, g_loss))

  if(epoch % save_interval) == 0:
    save_imgs(epoch)

  print(X_train.shape)

train(400, batch_size=64, save_interval=200)

************** 0 [D loss: 0.015769, acc: 99.22] [G loss: 6.794882]
************** 1 [D loss: 0.241749, acc: 92.19] [G loss: 3.790228]
************** 2 [D loss: 0.655665, acc: 78.91] [G loss: 1.864378]
************** 3 [D loss: 1.652482, acc: 64.06] [G loss: 0.549068]
************** 4 [D loss: 2.241443, acc: 56.25] [G loss: 0.174404]
************** 5 [D loss: 3.288042, acc: 50.78] [G loss: 0.039990]
************** 6 [D loss: 3.691968, acc: 50.78] [G loss: 0.007407]
************** 7 [D loss: 4.571404, acc: 49.22] [G loss: 0.011484]
************** 8 [D loss: 4.628678, acc: 50.00] [G loss: 0.004112]
************** 9 [D loss: 4.747078, acc: 49.22] [G loss: 0.001222]
************** 10 [D loss: 5.319050, acc: 50.00] [G loss: 0.001623]
************** 11 [D loss: 4.906177, acc: 50.00] [G loss: 0.008386]
************** 12 [D loss: 4.970658, acc: 50.78] [G loss: 0.000987]
************** 13 [D loss: 5.030145, acc: 50.00] [G loss: 0.001788]
************** 14 [D loss: 5.305889, acc: 50.00] [G loss: 

# 8) Making GIF

In [20]:
# Display a single image using the epoch number
# def display_image(epoch_no):
#   return PIL.Image.open('generated_images/%.8f.png'.format(epoch_no))

anim_file = 'scgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('generated_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)